In [1]:
import pandas as pd
import numpy as np
import scanpy as sc
import glob
import scanpypip.preprocessing as pp

In [2]:
f=glob.glob("data/GSE149383/raw/*D0*")

In [3]:
meta = pd.read_csv("data/GSE149383/raw/meta.csv")

In [4]:
f=glob.glob("data/GSE149383/raw/*GSM4494348*txt")[0]
f

'data/GSE149383/raw\\GSM4494348_D11C_expression_matrix.txt'

In [5]:
meta

,Run,Assay Type,AvgSpotLen,Bases,BioProject,BioSample,Bytes,Center Name,Consent,DATASTORE filetype,...,treatment,Cell_Line,GEO_Accession (exp),Replicate_number,Replicates,primary/metastaic,Replicate,smoking_history,stage,oncogenic_driver_mutation
0,SRR11597463,RNA-Seq,69,6561083228,PRJNA627676,SAMN14680068,3133079302,GEO,public,"fastq,sra",...,untreated,PC9,GSM4494347,biological replicate 2,NaN,NaN,NaN,NaN,NaN,NaN
1,SRR11597464,RNA-Seq,69,3163877900,PRJNA627676,SAMN14680067,1509616102,GEO,public,"fastq,sra",...,treated with 2 uM erlotinib,PC9,GSM4494348,biological replicate 1,NaN,NaN,NaN,NaN,NaN,NaN
2,SRR11597465,RNA-Seq,83,2307496801,PRJNA627676,SAMN14680066,994482575,GEO,public,"fastq,sra",...,treated with 2 uM erlotinib,PC9,GSM4494349,biological replicate 2,NaN,NaN,NaN,NaN,NaN,NaN
3,SRR11597466,RNA-Seq,83,6140858243,PRJNA627675,SAMN14680065,2840873219,GEO,public,"fastq,sra",...,treated with 2 uM erlotinib,PC9,GSM4494350,biological replicate 1,NaN,NaN,NaN,NaN,NaN,NaN
4,SRR11597467,RNA-Seq,83,6007124555,PRJNA627675,SAMN14679992,2766844368,GEO,public,"fastq,sra",...,treated with 2 uM erlotinib,PC9,GSM4494351,biological replicate 2,NaN,NaN,NaN,NaN,NaN,NaN
5,SRR11597468,RNA-Seq,83,4223410974,PRJNA627675,SAMN14679991,1946904146,GEO,public,"fastq,sra",...,treated with 1 uM erlotinib combined with 1 uM...,PC9,GSM4494352,biological replicate 1,NaN,NaN,NaN,NaN,NaN,NaN
6,SRR11597469,RNA-Seq,83,4356219790,PRJNA627675,SAMN14679990,2005258725,GEO,public,"fastq,sra",...,treated with 1 uM erlotinib combined with 1 uM...,PC9,GSM4494353,biological replicate 2,NaN,NaN,NaN,NaN,NaN,NaN


In [7]:
adatas = []
gen_sub = []

for idx ,row in meta.iterrows():
    accession = (row["GEO_Accession (exp)"])
    f=glob.glob("data/GSE149383/raw/*"+accession+"*txt")[0]
    adata = pp.read_sc_file(f,sep="\t")
    
    # Filter 
    sc.pp.filter_cells(adata, min_genes=200)
    sc.pp.filter_genes(adata, min_cells=3)
    
    if(len(gen_sub)==0):
        gen_sub = adata.var.index
    else:
        gen_sub = adata.var.index.intersection(gen_sub)
            
    print(adata.shape)
    
    adata.obs["treatment"] = row["treatment"]
    adata.obs["Replicate_number"] = row["Replicate_number"]
    adata.obs["Accession"] = accession

    adatas.append(adata)


D:\Anaconda3\lib\site-packages\anndata\_core\anndata.py:1094: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  if not is_categorical(df_full[k]):


(1204, 17376)
(586, 14618)
(1535, 14909)
(1365, 16548)
(1673, 17239)
(1526, 16133)
(1408, 15960)


In [8]:
for adata in adatas:
    adata = adata[:,gen_sub]

In [11]:
adatas[1:]

[AnnData object with n_obs × n_vars = 586 × 14618
     obs: 'n_genes', 'treatment', 'Replicate_number', 'Accession'
     var: 'n_cells', AnnData object with n_obs × n_vars = 1535 × 14909
     obs: 'n_genes', 'treatment', 'Replicate_number', 'Accession'
     var: 'n_cells', AnnData object with n_obs × n_vars = 1365 × 16548
     obs: 'n_genes', 'treatment', 'Replicate_number', 'Accession'
     var: 'n_cells', AnnData object with n_obs × n_vars = 1673 × 17239
     obs: 'n_genes', 'treatment', 'Replicate_number', 'Accession'
     var: 'n_cells', AnnData object with n_obs × n_vars = 1526 × 16133
     obs: 'n_genes', 'treatment', 'Replicate_number', 'Accession'
     var: 'n_cells', AnnData object with n_obs × n_vars = 1408 × 15960
     obs: 'n_genes', 'treatment', 'Replicate_number', 'Accession'
     var: 'n_cells']

In [13]:
adata_concat = adatas[0].concatenate(adatas[1:], batch_categories=meta["GEO_Accession (exp)"])

In [15]:
adata_concat.obs

,n_genes,treatment,Replicate_number,Accession,batch
GTTTGCTGTTTC-GSM4494347,5269,untreated,biological replicate 2,GSM4494347,GSM4494347
AGCAAATCACCG-GSM4494347,5397,untreated,biological replicate 2,GSM4494347,GSM4494347
CATGCCTGGTGT-GSM4494347,5463,untreated,biological replicate 2,GSM4494347,GSM4494347
CTTTAACCGCCT-GSM4494347,5262,untreated,biological replicate 2,GSM4494347,GSM4494347
CACCCCCAAGCA-GSM4494347,5094,untreated,biological replicate 2,GSM4494347,GSM4494347
...,...,...,...,...,...
GGTTCCCCTAGC-GSM4494353,211,treated with 1 uM erlotinib combined with 1 uM...,biological replicate 2,GSM4494353,GSM4494353
GCAAGAAAGCTC-GSM4494353,207,treated with 1 uM erlotinib combined with 1 uM...,biological replicate 2,GSM4494353,GSM4494353
GCTACCAAGTCG-GSM4494353,214,treated with 1 uM erlotinib combined with 1 uM...,biological replicate 2,GSM4494353,GSM4494353
GCCAACACCAAT-GSM4494353,202,treated with 1 uM erlotinib combined with 1 uM...,biological replicate 2,GSM4494353,GSM4494353


In [19]:
adata = adata_concat

In [20]:
sensitive = [int(row.find("untreated")==-1) for row in adata.obs.loc[:,"treatment"]]
adata.obs['sensitive'] = sensitive

sens_ = ['Resistant' if (row.find("untreated")==-1) else 'Sensitive' for row in adata.obs.loc[:,"treatment"]]
adata.obs['sensitivity'] = sens_


In [21]:
adata

AnnData object with n_obs × n_vars = 9297 × 12518
    obs: 'n_genes', 'treatment', 'Replicate_number', 'Accession', 'batch', 'sensitive', 'sensitivity'
    var: 'n_cells-GSM4494347', 'n_cells-GSM4494348', 'n_cells-GSM4494349', 'n_cells-GSM4494350', 'n_cells-GSM4494351', 'n_cells-GSM4494352', 'n_cells-GSM4494353'

In [24]:
adata.var

,n_cells-GSM4494347,n_cells-GSM4494348,n_cells-GSM4494349,n_cells-GSM4494350,n_cells-GSM4494351,n_cells-GSM4494352,n_cells-GSM4494353
GENE,,,,,,,
A2ML1,22,14,20,33,50,49,40
A4GALT,16,14,21,37,54,43,40
AAAS,43,23,41,62,91,69,60
AACS,100,35,88,72,118,84,90
AADAT,14,7,20,14,22,9,11
...,...,...,...,...,...,...,...
ZYX,183,117,170,328,497,305,319
ZZEF1,31,13,48,31,71,60,43
ZZZ3,88,63,83,86,110,114,107


In [25]:
adata.write("data/GSE149383.h5ad")